In [2]:
import pandas as pd
import numpy as np
import pickle

import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize, word_tokenize 

Idea: Match claims in reason dataset with IBM CoPA's by topic, then compare cosine similarities using avg W2V embeddings. Map topic to CoPA subject to embeddings, and one df to keep track of each topic in Reason.

In [3]:
pickle_copadict = open("./principle_argument_CoPA/PA_dict.pkl","rb")
principle_args = pickle.load(pickle_copadict)
print(principle_args)

{'Adolescent Rights': ['Many adolescents cannot make responsible decisions', 'Adolescents are as capable as adults'], 'Animal Rights': ['Animals should not be treated as property', 'There is nothing wrong with using animals to further human interests'], 'Big government': ['Public utility is best served by actions coordinated by central government', 'Public interest is best served and propelled by voluntary interactions,and not ones dictated by government'], 'Black market': ['Prohibiting products and activities makes them less visible and available, and thus less harmful', 'Prohibition is counterproductive and only leads to increased demand'], 'Clean energy': ['Humanity must embrace clean energy in order to fight climate change', 'Ecological concerns add further strain on the economy'], 'Coercion': ['A decisive and enforced policy is the best way to deliver a message', 'Enforcement tends to be less effective than persuasion and education'], 'Conservatism': ['The current system is workin

In [4]:
copa_df = pd.read_csv("./principle_argument_CoPA/IBM_Debater_(R)_CoPA-Motion-ACL-2019.v0.csv")
copa_df

,Action,Topic,Wikipedia title,Motion phrasing,Initial list?,# CoPAs,Adolescent rights,Animal rights,Big government,Black market,...,Right to privacy,Self-determination,Sexual morality,Subsidies,Technology,Tradition,Value of science,Virtual life,Wealth distribution,Welfare state
0,mandatory,Abstinence-only sex education,Abstinence-only sex education,Abstinence-only sex education should be mandatory,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,end,affirmative action,Affirmative action,We should end affirmative action,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,increase,airport racial profiling in the United States,Airport racial profiling in the United States,We should increase airport racial profiling in...,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,ban,alcoholic beverages,Alcoholic drink,We should ban alcoholic beverages,1.0,4.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,abolish,the American Jobs Act,American Jobs Act,We should abolish the American Jobs Act,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# copa_df['CoPA']['same sex marriage']
matching_topics=['abortions', 'same sex marriage','the foreign policy of the Barack Obama administration']
matching_copa_df = copa_df.loc[copa_df['Topic'].isin(matching_topics)]
matching_copa_df

,Action,Topic,Wikipedia title,Motion phrasing,Initial list?,# CoPAs,Adolescent rights,Animal rights,Big government,Black market,...,Right to privacy,Self-determination,Sexual morality,Subsidies,Technology,Tradition,Value of science,Virtual life,Wealth distribution,Welfare state
78,legalize,same sex marriage,Same-sex marriage,We should legalize same sex marriage,1.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-1.0,0.0,0,-1.0,0.0,0.0,0.0,0.0
101,ban,abortions,Abortion,We should ban abortions,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
306,not abandon,the foreign policy of the Barack Obama adminis...,Foreign policy of the Barack Obama administration,We should not abandon the foreign policy of th...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [6]:
matching_copa_df

,Action,Topic,Wikipedia title,Motion phrasing,Initial list?,# CoPAs,Adolescent rights,Animal rights,Big government,Black market,...,Right to privacy,Self-determination,Sexual morality,Subsidies,Technology,Tradition,Value of science,Virtual life,Wealth distribution,Welfare state
78,legalize,same sex marriage,Same-sex marriage,We should legalize same sex marriage,1.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-1.0,0.0,0,-1.0,0.0,0.0,0.0,0.0
101,ban,abortions,Abortion,We should ban abortions,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
306,not abandon,the foreign policy of the Barack Obama adminis...,Foreign policy of the Barack Obama administration,We should not abandon the foreign policy of th...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [7]:
matching_copa_df == 1

,Action,Topic,Wikipedia title,Motion phrasing,Initial list?,# CoPAs,Adolescent rights,Animal rights,Big government,Black market,...,Right to privacy,Self-determination,Sexual morality,Subsidies,Technology,Tradition,Value of science,Virtual life,Wealth distribution,Welfare state
78,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
101,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
# (matching_copa_df == 1).iloc[0]

In [9]:
copa_cols = matching_copa_df.loc[:, 'Adolescent rights':'Welfare state']

In [10]:
from collections import defaultdict
topic_idx_to_copa = defaultdict(list)
for i in range(3):
    topic_idx_to_copa[i].append(copa_cols.columns[(copa_cols == 1).iloc[i]].tolist())
    topic_idx_to_copa[i].append(copa_cols.columns[(copa_cols == -1).iloc[i]].tolist())

In [11]:
topic_idx_to_copa

defaultdict(list,
            {0: [[], ['Discrimination', 'Sexual morality', 'Tradition']],
             1: [['Coercion'], ['Freedom of choice', 'Playing god']],
             2: [[], []]})

In [12]:
reasons_topic_to_copa_subject={'gayRights':['Discrimination', 'Sexual morality', 'Tradition'],
                      'abortion':['Coercion','Freedom of choice', 'Playing god']
                      }

In [13]:
import pathlib
pathlib.Path().absolute()

PosixPath('/Users/joshy/cs577/577project-gitrepo/data')

In [14]:
wv_from_bin = KeyedVectors.load_word2vec_format("./word2vec/GoogleNews-vectors-negative300.bin",limit=50000, binary=True)

In [15]:
copa_subject_to_sent = defaultdict(list)
for key, val in reasons_topic_to_copa_subject.items():
    for topic in val:
        # two arguments per topic
        arguments = principle_args[topic]
        copa_subject_to_sent[topic]=(arguments)

In [16]:
# each subject has two sentences
copa_subject_to_sent['Discrimination']

['It is a fact that there are differences between people. Hence, there should sometimes be differences in the way people are treated.',
 'All people should be treated equally']

In [17]:
def get_avg_vector(w2v_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in w2v_model.vocab]
    if len(words) >= 1:
        return np.mean(w2v_model[words], axis=0)
    else:
        return []

In [18]:
def embed_sentence(s):
    if s[-1] == '.':
        s=s[:-1]
    data = [] 
    words = s.split()
    avg = get_avg_vector(wv_from_bin, words)
    return avg
# embed_sentence('It is a fact that there are differences between people. Hence, there should sometimes be differences in the way people are treated.')

In [19]:
# creating dictionary subject to two avg sentence embeddings
copa_subject_to_embeds = defaultdict(list)
for subject, pair in copa_subject_to_sent.items():
    copa_subject_to_embeds[subject]=[]
    for sent in pair:
        copa_subject_to_embeds[subject].append(embed_sentence(sent))
        print(len(copa_subject_to_embeds[subject]))

1
2
1
2
1
2
1
2
1
2
1
2


In [20]:
len(copa_subject_to_embeds['Playing god'])

2

We map topic to copa subject to embeds

In [21]:
reasons_topic_to_copa_subject['abortion']

['Coercion', 'Freedom of choice', 'Playing god']

In [22]:
copa_subject_to_embeds

defaultdict(list,
            {'Discrimination': [array([ 0.03961985,  0.03656327,  0.08225612,  0.10595703, -0.12070023,
                      0.02340538,  0.04561896, -0.06718364,  0.07693452,  0.12739001,
                     -0.07481625, -0.10284745, -0.03426642,  0.07345742, -0.09658974,
                      0.10922241,  0.06132748,  0.14956826, -0.04761947, -0.04589201,
                     -0.07482499,  0.01710912,  0.03368498, -0.04113328,  0.01691477,
                     -0.01996974, -0.11597081,  0.04055947, -0.04398466, -0.01872173,
                     -0.09973787, -0.00774825,  0.003373  ,  0.01788973,  0.06205027,
                      0.05844598,  0.04492188,  0.0202958 ,  0.06962706,  0.02791716,
                      0.12531482,  0.03789159,  0.02767784,  0.00058786, -0.04397422,
                     -0.02218026, -0.0518317 ,  0.04503752, -0.06499562, -0.04327714,
                      0.00177966,  0.0680285 , -0.04218172, -0.02457468,  0.04163401,
                  

In [23]:
reasons_df = pd.read_csv("./reason-dataset.csv")
reasons_df

,id,topic,stance,substance,source,line
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ..."
...,...,...,...,...,...,...
2843,2843,obama,Pro,p-quality,B64,"While I wish he would have accomplished more, ..."
2844,2844,obama,Pro,p-republicans,B64,Obama inherited many of the deficiencies lefto...
2845,2845,obama,Con,c-War,F38,He has gotten troops out of countries Bush...
2846,2846,obama,Pro,p-economy,K33,fincial reform- made the banks do their buince...


In [24]:
# marijuana is not a CoPA topic so we don't care about it here
reasons_df = reasons_df[reasons_df['topic'] != 'marijuana']

In [25]:
reasons_df_abort = reasons_df[reasons_df['topic'] == 'abortion']
reasons_df_gay = reasons_df[reasons_df['topic'] == 'gayRights']
reasons_df_obama = reasons_df[reasons_df['topic'] == 'obama']

In [26]:
# Convert each sentence into sentence embedding
abort_embeds = []
for sentence in reasons_df_abort['line']:
    emb = embed_sentence(sentence)
#     print(emb)
    abort_embeds.append(emb)
gay_embeds = []
for sentence in reasons_df_gay['line']:
    emb = embed_sentence(sentence)
#     print(emb)
    gay_embeds.append(emb)
obama_embeds = []
for sentence in reasons_df_obama['line']:
    emb = embed_sentence(sentence)
#     print(emb)
    obama_embeds.append(emb)


In [27]:
np.asarray(abort_embeds).shape

(739, 300)

In [28]:
# reasons_df_abort = reasons_df_abort.assign(e=np.asarray(abort_embeds))
reasons_df_obama.head()

,id,topic,stance,substance,source,line
2202,2202,obama,Pro,p-health,M25,"Imagine a three child, out of work family that..."
2203,2203,obama,Con,c-Other,F87,"But if you believe in freedom, and the Constit..."
2204,2204,obama,Pro,p-republicans,G2,You can't expect shit to get done right now ....
2205,2205,obama,Pro,p-War,F15,Ended War in Iraq. Self-Explanotry.
2206,2206,obama,Pro,p-quality,F15,Received the Nobel Peace Prize. Now this isn't...


In [29]:
reasons_df_abort.loc[:,'embeds'] = pd.Series(abort_embeds,index=reasons_df_abort.index)
reasons_df_gay.loc[:,'embeds'] = pd.Series(gay_embeds,index=reasons_df_gay.index)
reasons_df_obama.loc[:,'embeds'] = pd.Series(obama_embeds,index=reasons_df_obama.index)

/Users/joshy/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/joshy/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [30]:
# reasons_df_abort.head()
# reasons_df_gay.head()
reasons_df_obama.head()

,id,topic,stance,substance,source,line,embeds
2202,2202,obama,Pro,p-health,M25,"Imagine a three child, out of work family that...","[0.022695541, 0.040164307, 0.0637722, 0.075544..."
2203,2203,obama,Con,c-Other,F87,"But if you believe in freedom, and the Constit...","[0.08790499, 0.04086663, 0.08885282, 0.1102797..."
2204,2204,obama,Pro,p-republicans,G2,You can't expect shit to get done right now ....,"[0.07169526, 0.06985684, 0.041292913, 0.129070..."
2205,2205,obama,Pro,p-War,F15,Ended War in Iraq. Self-Explanotry.,"[0.1529948, -0.00064086914, -0.03108724, 0.050..."
2206,2206,obama,Pro,p-quality,F15,Received the Nobel Peace Prize. Now this isn't...,"[0.027088512, 0.03314885, 0.097317785, 0.11730..."


In [31]:
reasons_topic_to_copa_subject

{'gayRights': ['Discrimination', 'Sexual morality', 'Tradition'],
 'abortion': ['Coercion', 'Freedom of choice', 'Playing god']}

In [32]:
len(copa_subject_to_embeds['Discrimination']) #list of two embeddings

2

In [33]:
len(copa_subject_to_embeds['Coercion']) #list of two embeddings

2

In [34]:
gay_rights_embeds = []
for copa_subject in reasons_topic_to_copa_subject['gayRights']:
    embed_pair = copa_subject_to_embeds[copa_subject]
    gay_rights_embeds.extend(embed_pair)
abortion_embeds = []
for copa_subject in reasons_topic_to_copa_subject['abortion']:
    embed_pair = copa_subject_to_embeds[copa_subject]
    abortion_embeds.extend(embed_pair)

In [35]:
len(gay_rights_embeds) == 6
len(abortion_embeds) == 6

True

In [36]:
abortion_embeds = np.asarray(abortion_embeds)
gay_rights_embeds = np.asarray(gay_rights_embeds)

In [37]:
abortion_embeds[0].shape

(300,)

In [38]:
reasons_df_abort.embeds[0].shape

(300,)

In [42]:
CoPAs = pd.read_pickle(r'principle_argument_CoPA/PA_list.pkl')
CoPAs[:2]

['Many adolescents cannot make responsible decisions',
 'Adolescents are as capable as adults']

In [43]:
# sentences = claims_train.values.tolist()
# sentences.extend(CoPAs)
# infersent.update_vocab(sentences)

# Embedding all CoPAs
CoPAs_embedding = []
for arg in CoPAs:
    CoPAs_embedding.append(embed_sentence(arg))

In [45]:
len(CoPAs_embedding)

74

In [39]:
cosine_similarity(reasons_df_abort.embeds[0],abortion_embeds[0])

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.03436127  0.03716803  0.01741333  0.14700623 -0.11905365  0.01825828
  0.0802742  -0.02402878  0.06729736  0.07179298 -0.0606842  -0.1266922
 -0.03468018  0.06435547 -0.12084655  0.12060241  0.07551269  0.10825195
 -0.02239161 -0.03613205 -0.02010345 -0.00812683  0.0790268  -0.06903686
  0.00272131 -0.0418335  -0.09134521  0.05032654  0.00811653 -0.04252014
  0.00374451 -0.00116272 -0.04248047 -0.06499939 -0.01664429 -0.02218933
  0.03138123 -0.00661469  0.03086243  0.00905762  0.11025391 -0.01368408
  0.09689446 -0.06484985 -0.02338829 -0.03631897 -0.00609055  0.07542725
 -0.01410999 -0.0043457   0.00039444  0.04322357 -0.02982788 -0.01478271
  0.00423584  0.00619278 -0.00834579 -0.09883881  0.06340637  0.0072464
  0.00189209  0.01816864 -0.09424438 -0.0441803   0.00257339 -0.02852478
 -0.05669212  0.09605408 -0.08215103  0.05345993  0.06118164  0.05245972
  0.0503891   0.07824783 -0.10145263 -0.04154968  0.06228027  0.124

In [ ]:
# similarities = [cosine_similarity(reasons_df_abort.embeds[0],pa_embed) for pa_embed in abortion_embeds]
# similarities

In [ ]:
for row in reasons_df_abort.itertuples():
    max_cosine_similarity=0
    closest_pa = 0
    for i in range(len(abortion_embeds)):
        pa_embed = abortion_embeds[i]
        similarity =  cosine_similarity(row.embeds,pa_embed)
        if similarity > max_cosine_similarity:
            max_cosine_similarity = similarity
            closest_pa = i
    print(closest_pa)
    # convert to np arrays
#     similarities = [cosine_similarity(row.embeds,pa_embed) for pa_embed in abortion_embeds]
    
    print(row.Index)

In [ ]:
#given sentence find pa with smallest cosine similarity




In [ ]:
embed1 = reasons_df_abort['embeds'][0]

In [ ]:
embed1